In [2]:
datalist = list()
with open("inst6502_comp.txt", "r") as f:
    for line in f:
        data = line.strip().split('\t')
        datalist.extend([i.split(" ",1) for i in data])

datatypes = list(set([i[1] for i in datalist]))
print(datatypes)
#print(len(datatypes))
with open("../debug_itab.z80", "w") as f:
    f.write("inst6502_opcode_width_table:\n")
    for idx, i in enumerate(datatypes):
        size = 0
        if i in ("", "impl", "A"):
            size = 1
        elif i in ("zpg", "rel", "X,ind", "ind,Y", "#", "zpg,X", "zpg,Y"):
            size = 2
        elif i in ("abs", "abs,X", "abs,Y", "ind"):
            size = 3
        pass
        s = f".db {size}    ;{idx}: {i}\n"
        f.write(s)
    f.write("\n\n;Note: 1=1byteOper, 2=2byteOper, -=1byteRelOper. Other chrs are literal\n")
    f.write("\ninst6502_disassembler:\n")
    for idx, i in enumerate(datatypes):
        if i == "":
            s = ""
        elif i == "impl":
            s = ""
        elif i == "A":
            s = "A"
        elif i == "#":
            s = "#1"
        elif i == "zpg":
            s = "1"
        elif i == "rel":
            s = "-"
        elif i == "zpg,X":
            s = "1,X"
        elif i == "zpg,Y":
            s = "1,Y"
        elif i == "X,ind":
            s = "(1,X)"
        elif i == "ind,Y":
            s = "(1),Y"
        elif i == "ind":
            s = "(2)"
        elif i == "abs":
            s = "2"
        elif i == "abs,X":
            s = "2,X"
        elif i == "abs,Y":
            s = "2,Y"
        size = len(s)+1
        f.write(f'.db {size},"{s}"  ;\n')
        


    f.write("\ninst6502_disassembly_table:\n")
    for idx, i in enumerate(datalist):
        s = f'.db "{i[0].strip()[:3]}", {datatypes.index(i[1])}     ;0x{idx:02X}\n'
        f.write(s)





['', 'zpg,X', 'ind', 'abs', 'abs,X', 'X,ind', 'A', 'zpg', 'rel', '#', 'impl', 'ind,Y', 'abs,Y', 'zpg,Y']


In [9]:
import hashlib, os, importlib
import toolkit
importlib.reload(toolkit)

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


filename = "nestest.nes"
filebase = filename.split('.')[0]
with open("nestest.nes","rb") as f:
    data = bytes(f.read())

print(data[:15])
isrom = False
def lo(num):
    return num & 0xFF
def hi(num):
    return (num >> 8) & 0xFF
if bytes(data[:4]) == bytes([ord('N'),ord('E'),ord('S'), 0x1A]):
    print("Is NES ROM")
    isrom = True
    prg_rom_count = int(data[4])
    chr_rom_count = int(data[5])
    print(f"PRG COUNT {prg_rom_count}, CHR COUNT {chr_rom_count}")
    hashdata = hashlib.sha256(data).digest()[:16]
    print(hashdata)
    datalen = len(data)
    convdata = tobytes(["CENROM", 0, filebase, 0, hashdata, 0, lo(datalen), hi(datalen), data])
    toolkit.export8xv(filebase, convdata)
    if os.path.exists(f"../bin/{filebase}.8xv"):
        os.remove(f"../bin/{filebase}.8xv")
    os.rename(f"{filebase}.8xv",f"../bin/{filebase}.8xv")
    a = (data[6] & 0xF0) >> 4
    b = (data[7] & 0xF0)
    print(f"Mapper ID: {a|b}")



if not isrom:
    print("Failed an NES ROM check.")



b'NES\x1a\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Is NES ROM
PRG COUNT 1, CHR COUNT 1
b'\xf6}U\xfdk<\xf0\xba\xd1\xcc\x85\xf1\xdf\rs\x9c'
Mapper ID: 0


In [7]:
import hashlib, os, importlib
import toolkit

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


timing_list = list()
status_list = list()
with open("nestest.log","r") as f:
    for line in f:
        linedata = line.split("CYC:")[1].strip()
        statusdata = line.split("P:")[1].split()[0]
        if linedata:
            status_list.append(int(statusdata, base=16))
            timing_list.append(int(linedata))
print(len(timing_list))
timing_array = list()
for sitem,item in zip(status_list,timing_list):
    timing_array.append(item&0xFF)
    timing_array.append((item>>8)&0xFF)
    timing_array.append(sitem)

filebase = "timings"

toolkit.export8xv(filebase, timing_array)
if os.path.exists(f"../bin/{filebase}.8xv"):
    os.remove(f"../bin/{filebase}.8xv")
os.rename(f"{filebase}.8xv",f"../bin/{filebase}.8xv")


8991


In [5]:
import hashlib, os, importlib
import toolkit
importlib.reload(toolkit)

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


filename = "nestest.nes"
filebase = filename.split('.')[0]
with open("nestest.nes","rb") as f:
    data = bytes(f.read())

print(data[:15])
isrom = False
def lo(num):
    return num & 0xFF
def hi(num):
    return (num >> 8) & 0xFF
if bytes(data[:4]) == bytes([ord('N'),ord('E'),ord('S'), 0x1A]):
    print("Is NES ROM")
    isrom = True
    prg_rom_count = int(data[4])
    chr_rom_count = int(data[5])
    print(f"PRG COUNT {prg_rom_count}, CHR COUNT {chr_rom_count}")
    hashdata = hashlib.sha256(data).digest()[:16]
    print(hashdata)
    for i in range(0xC6E8,0xC700):
        print(f"{i:04X}: {data[i+16-0xC000]:02X}")

b'NES\x1a\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Is NES ROM
PRG COUNT 1, CHR COUNT 1
b'\xf6}U\xfdk<\xf0\xba\xd1\xcc\x85\xf1\xdf\rs\x9c'
C6E8: DA
C6E9: FA
C6EA: 80
C6EB: 89
C6EC: EA
C6ED: EA
C6EE: EA
C6EF: EA
C6F0: 08
C6F1: 48
C6F2: 1C
C6F3: A9
C6F4: A9
C6F5: 3C
C6F6: A9
C6F7: A9
C6F8: 5C
C6F9: A9
C6FA: A9
C6FB: 7C
C6FC: A9
C6FD: A9
C6FE: DC
C6FF: A9


In [5]:
import os, struct


#note: implementation detail sourced from graphx.h
#((uint16_t)(((((r) & 255) >> 3) << 10) | ((((g) & 255) >> 3) << 5) | (((b) & 255) >> 3)))

def convrgbto1555(rgbtuple:list|tuple):
    r,g,b = rgbtuple
    return (((((r) & 255) >> 3) << 10) | ((((g) & 255) >> 3) << 5) | (((b) & 255) >> 3))

filedata = list()
with open("Composite_wiki.pal","rb") as f:
    filebin = f.read()
filedata = [[r,g,b] for r,b,g in struct.iter_unpack("BBB",filebin)]
print(filedata)

filedata = [convrgbto1555(i) for i in filedata]
linelen = 8
filedata = [filedata[n:n+linelen] for n in range(0,len(filedata), linelen)]

s = "paletteData:\n"

for line in filedata:
    s += ".dw "+', '.join([f'${i:04X}' for i in line])+"\n"


with open("../palette.z80", "w") as f:
    f.write(s)




[[98, 98, 98], [0, 124, 44], [17, 156, 21], [54, 156, 3], [85, 124, 0], [103, 68, 0], [103, 3, 7], [85, 0, 28], [54, 0, 50], [17, 0, 68], [0, 0, 78], [0, 3, 76], [0, 68, 64], [0, 0, 0], [0, 0, 0], [0, 0, 0], [171, 171, 171], [18, 206, 96], [61, 250, 66], [110, 250, 41], [153, 206, 28], [177, 129, 30], [177, 41, 47], [153, 0, 74], [110, 0, 105], [61, 0, 130], [18, 0, 143], [0, 41, 141], [0, 129, 124], [0, 0, 0], [0, 0, 0], [0, 0, 0], [255, 255, 255], [96, 255, 178], [141, 255, 146], [192, 255, 120], [236, 255, 106], [255, 212, 109], [255, 121, 127], [236, 42, 155], [192, 0, 186], [141, 0, 212], [96, 42, 226], [71, 121, 224], [71, 212, 206], [78, 78, 78], [0, 0, 0], [0, 0, 0], [255, 255, 255], [191, 255, 224], [209, 255, 211], [230, 255, 201], [247, 255, 195], [255, 238, 196], [255, 201, 203], [247, 169, 215], [230, 151, 227], [209, 151, 238], [191, 169, 243], [181, 201, 242], [181, 238, 235], [184, 184, 184], [0, 0, 0], [0, 0, 0], [100, 70, 71], [0, 92, 21], [18, 123, 3], [53, 125, 0], 